In [1]:
import sys
import os
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import VotingClassifier
from sklearn.base import clone 
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import (
    confusion_matrix,      # To calculate the confusion matrix
    balanced_accuracy_score,  # To calculate balanced accuracy
    accuracy_score,         # To calculate overall accuracy
    precision_score,        # To calculate precision (macro and binary)
    recall_score,           # To calculate recall (macro and binary)
    f1_score,               # To calculate F1-score (macro and binary)
    roc_auc_score,          # To calculate the Area Under the ROC Curve (AUC-ROC)
    roc_curve,              # To generate the ROC curve (false positive vs. true positive rates)
    precision_recall_curve  # To generate Precision-Recall curves
)
import time  # To measure time complexity if needed
# import eli5
# from eli5.sklearn import PermutationImportance

In [3]:
data = pd.read_csv('transaction_dataset.csv')
device='cuda:0' if torch.cuda.is_available() else 'cpu'

In [4]:
df = data.drop(columns=['is_exchange', 'cluster_label', 'timestamp', 'transaction_hash'])

In [5]:
df

,transaction_index,block_hash,block_height,block_time,historical_usd_price,from,from_chain,to,to_chain,type,indegree,outdegree,in_btc,out_btc,total_btc,mean_in_btc,mean_out_btc
0,NaN,00000000000000000001e10cc414e3a7e8a3c4b4ac6ab0...,872509.0,2024-11-29T19:55:58Z,9.794225e+06,3D1xsYjkitAQR3MmgP5xyvs3vKNSiu7vot,bitcoin,3GVyAw5LX5uP1wDQgKtL1e3cPni4cXqkac,bitcoin,NaN,10,5,10067766067,10067750518,20135516585,1.006777e+09,2.013550e+09
1,NaN,000000000000000000025b187a8d14865fce4f03c798dd...,872504.0,2024-11-29T19:08:21Z,1.782262e+06,bc1qa3phj5uhnuauk6r62cku6r6fl9rawqx4n6d690,bitcoin,3D1xsYjkitAQR3MmgP5xyvs3vKNSiu7vot,bitcoin,NaN,1,2,1833508472,1833507279,3667015751,1.833508e+09,9.167536e+08
2,NaN,000000000000000000021149189c5cb77924856986570f...,872494.0,2024-11-29T17:30:07Z,1.099590e+07,bc1qvsakx7kuwdey8mewnn6p835camt24p5vnnz3mpa2hn...,bitcoin,bc1qtcqmrlsjrk6sl8arwytvv5wtkwyu0028us4g3e,bitcoin,NaN,31,25,11313119464,11313038365,22626157829,3.649393e+08,4.525215e+08
3,NaN,00000000000000000001a3e2efc3537cc9345a07ada612...,872491.0,2024-11-29T17:10:22Z,9.452636e+05,bc1qm34lsc65zpw79lxes69zkqmk6ee3ewf0j77s3h,bitcoin,bc1qx92rmjeuqj38u0qaeqf2u3ydvrvyv70fxrgq5e,bitcoin,NaN,1,20,965776741,965765791,1931542532,9.657767e+08,4.828829e+07
4,NaN,0000000000000000000164f939cf5b727824101b04bf08...,872489.0,2024-11-29T17:03:50Z,1.169275e+07,3HcEUguUZ4vyyMAPWDPUDjLqz882jXwMfV,bitcoin,bc1q952ge707vx0ewzsc5erymxzm9uymfvw2arvl8e,bitcoin,NaN,5,23,11946497041,11946465251,23892962292,2.389299e+09,5.194115e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,NaN,000000000000000000020d948fccb83378c9dbf36b2362...,838057.0,2024-04-07T02:24:31Z,8.938877e+05,1NBSAB1xtgN5BUbUJJtoe8ejySzmqwBjHG,bitcoin,3D1xsYjkitAQR3MmgP5xyvs3vKNSiu7vot,bitcoin,NaN,2,2,1285060000,1285055076,2570115076,6.425300e+08,6.425275e+08
995,NaN,000000000000000000016e7b312f1a0a57f56b6dfa35ab...,838041.0,2024-04-06T22:48:45Z,6.875000e+05,3D1xsYjkitAQR3MmgP5xyvs3vKNSiu7vot,bitcoin,bc1qjh0xk3wzpj902lz32cd7aatv7j5744sn4xcxvv,bitcoin,NaN,1,5,1000000000,999995893,1999995893,1.000000e+09,1.999992e+08
996,NaN,00000000000000000002610acd6c670265feb2dfb603e7...,837989.0,2024-04-06T16:12:20Z,8.364064e+05,bc1qu7pelg829ecfw83unluw937cqcph0k0sm4lzjct99n...,bitcoin,bc1q8ef9vvyq6sj9c3gykqtswvfdrxrk6kz92rwtse,bitcoin,NaN,6,8,1228221249,1228206493,2456427742,2.047035e+08,1.535258e+08
997,NaN,00000000000000000000c68826329b64539237d644fb92...,837987.0,2024-04-06T16:00:50Z,5.619979e+05,3Aib5Rm9PrE9Kaptb7c2tPFuDf147Pbs9m,bitcoin,bc1q5hzx3khyzfky74myxf7c36dlfa7egcw97geap5,bitcoin,NaN,3,10,824696848,824684098,1649380946,2.748989e+08,8.246841e+07


In [6]:
scaled_df = df.copy()

scaled_df['indegree'] = np.log1p(scaled_df['indegree'])
scaled_df['outdegree']= np.log1p(scaled_df['outdegree'])
scaled_df['in_btc'] = np.log1p(scaled_df['in_btc'])
scaled_df['out_btc']= np.log1p(scaled_df['out_btc'])
scaled_df['total_btc']= np.log1p(scaled_df['total_btc'])
scaled_df['mean_in_btc']= np.log1p(scaled_df['mean_in_btc'])
scaled_df['mean_out_btc']= np.log1p(scaled_df['mean_out_btc'])

In [7]:
scaled_df

,transaction_index,block_hash,block_height,block_time,historical_usd_price,from,from_chain,to,to_chain,type,indegree,outdegree,in_btc,out_btc,total_btc,mean_in_btc,mean_out_btc
0,NaN,00000000000000000001e10cc414e3a7e8a3c4b4ac6ab0...,872509.0,2024-11-29T19:55:58Z,9.794225e+06,3D1xsYjkitAQR3MmgP5xyvs3vKNSiu7vot,bitcoin,3GVyAw5LX5uP1wDQgKtL1e3cPni4cXqkac,bitcoin,NaN,2.397895,1.791759,23.032605,23.032603,23.725751,20.730020,21.423165
1,NaN,000000000000000000025b187a8d14865fce4f03c798dd...,872504.0,2024-11-29T19:08:21Z,1.782262e+06,bc1qa3phj5uhnuauk6r62cku6r6fl9rawqx4n6d690,bitcoin,3D1xsYjkitAQR3MmgP5xyvs3vKNSiu7vot,bitcoin,NaN,0.693147,1.098612,21.329497,21.329497,22.022644,21.329497,20.636349
2,NaN,000000000000000000021149189c5cb77924856986570f...,872494.0,2024-11-29T17:30:07Z,1.099590e+07,bc1qvsakx7kuwdey8mewnn6p835camt24p5vnnz3mpa2hn...,bitcoin,bc1qtcqmrlsjrk6sl8arwytvv5wtkwyu0028us4g3e,bitcoin,NaN,3.465736,3.258097,23.149229,23.149222,23.842373,19.715242,19.930346
3,NaN,00000000000000000001a3e2efc3537cc9345a07ada612...,872491.0,2024-11-29T17:10:22Z,9.452636e+05,bc1qm34lsc65zpw79lxes69zkqmk6ee3ewf0j77s3h,bitcoin,bc1qx92rmjeuqj38u0qaeqf2u3ydvrvyv70fxrgq5e,bitcoin,NaN,0.693147,3.044522,20.688443,20.688432,21.381585,20.688443,17.692700
4,NaN,0000000000000000000164f939cf5b727824101b04bf08...,872489.0,2024-11-29T17:03:50Z,1.169275e+07,3HcEUguUZ4vyyMAPWDPUDjLqz882jXwMfV,bitcoin,bc1q952ge707vx0ewzsc5erymxzm9uymfvw2arvl8e,bitcoin,NaN,1.791759,3.178054,23.203704,23.203701,23.896850,21.594266,20.068207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,NaN,000000000000000000020d948fccb83378c9dbf36b2362...,838057.0,2024-04-07T02:24:31Z,8.938877e+05,1NBSAB1xtgN5BUbUJJtoe8ejySzmqwBjHG,bitcoin,3D1xsYjkitAQR3MmgP5xyvs3vKNSiu7vot,bitcoin,NaN,1.098612,1.098612,20.974071,20.974067,21.667217,20.280924,20.280920
995,NaN,000000000000000000016e7b312f1a0a57f56b6dfa35ab...,838041.0,2024-04-06T22:48:45Z,6.875000e+05,3D1xsYjkitAQR3MmgP5xyvs3vKNSiu7vot,bitcoin,bc1qjh0xk3wzpj902lz32cd7aatv7j5744sn4xcxvv,bitcoin,NaN,0.693147,1.791759,20.723266,20.723262,21.416411,20.723266,19.113824
996,NaN,00000000000000000002610acd6c670265feb2dfb603e7...,837989.0,2024-04-06T16:12:20Z,8.364064e+05,bc1qu7pelg829ecfw83unluw937cqcph0k0sm4lzjct99n...,bitcoin,bc1q8ef9vvyq6sj9c3gykqtswvfdrxrk6kz92rwtse,bitcoin,NaN,1.945910,2.197225,20.928833,20.928821,21.621974,19.137073,18.849379
997,NaN,00000000000000000000c68826329b64539237d644fb92...,837987.0,2024-04-06T16:00:50Z,5.619979e+05,3Aib5Rm9PrE9Kaptb7c2tPFuDf147Pbs9m,bitcoin,bc1q5hzx3khyzfky74myxf7c36dlfa7egcw97geap5,bitcoin,NaN,1.386294,2.397895,20.530526,20.530511,21.223666,19.431914,18.227926


In [8]:
rob_scaler = RobustScaler()

In [9]:
scaled_df['indegree'] = rob_scaler.fit_transform(scaled_df['indegree'].values.reshape(-1,1))
scaled_df['outdegree'] = rob_scaler.fit_transform(scaled_df['outdegree'].values.reshape(-1,1))
scaled_df['in_btc'] = rob_scaler.fit_transform(scaled_df['in_btc'].values.reshape(-1,1))
scaled_df['out_btc'] = rob_scaler.fit_transform(scaled_df['out_btc'].values.reshape(-1,1))
scaled_df['total_btc'] = rob_scaler.fit_transform(scaled_df['total_btc'].values.reshape(-1,1))
scaled_df['mean_in_btc'] = rob_scaler.fit_transform(scaled_df['mean_in_btc'].values.reshape(-1,1))
scaled_df['mean_out_btc'] = rob_scaler.fit_transform(scaled_df['mean_out_btc'].values.reshape(-1,1))

In [12]:
def evaluate(actual, predicted, probability, time_complexity):
    eval_metric = {}
    eval_metric['time'] = time_complexity
    eval_metric['conf_matrix'] = confusion_matrix(actual, predicted)
    
    eval_metric['balanced_accuracy'] = round(balanced_accuracy_score(actual, predicted),7)
    eval_metric['accuracy'] = round(accuracy_score(actual, predicted),7)
    
    eval_metric['macro_precision'] = round(precision_score(actual, predicted, average="macro"),7)
    eval_metric['macro_recall'] = round(recall_score(actual,predicted, average="macro"),7)
    eval_metric['macro_f1'] = round(f1_score(actual,predicted, average="macro"),7)
    eval_metric['macro_roc'] = round(roc_auc_score(actual, probability, average="macro"),7)

    eval_metric['precision'] = round(precision_score(actual, predicted),7)
    eval_metric['recall'] = round(recall_score(actual,predicted),7)
    eval_metric['f1'] = round(f1_score(actual,predicted),7)
    eval_metric['roc'] = round(roc_auc_score(actual, probability),7)

    false_pos_rate, true_pos_rate, thresholds = roc_curve(actual, probability)
    eval_metric['false_pos_rate'] = false_pos_rate
    eval_metric['true_pos_rate'] = true_pos_rate
    eval_metric['thresholds'] = thresholds
    
    precision_rt, recall_rt, threshold_rt = precision_recall_curve(actual, probability)
    eval_metric['precision_rt'] = precision_rt
    eval_metric['recall_rt'] = recall_rt
    eval_metric['threshold_rt'] = threshold_rt
    return eval_metric

In [ ]:
#Updated Current Model
from __future__ import division
from __future__ import print_function
from tqdm import tqdm

import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_array
from sklearn.utils.validation import check_is_fitted

from pyod.utils.utility import check_parameter
from pyod.utils.stat_models import pairwise_distances_no_broadcast

from pyod.models.base import BaseDetector

class AutoEncoder(BaseDetector):
    """Auto Encoder (AE) is a type of neural networks for learning useful data
    representations unsupervisedly. Similar to PCA, AE could be used to
    detect outlying objects in the data by calculating the reconstruction
    errors. See :cite:`aggarwal2015outlier` Chapter 3 for details.

    Parameters
    ----------
    hidden_neurons : list, optional (default=[64, 32, 32, 64])
        The number of neurons per hidden layers.

    hidden_activation : str, optional (default='relu')
        Activation function to use for hidden layers.
        All hidden layers are forced to use the same type of activation.
        See https://keras.io/activations/

    output_activation : str, optional (default='sigmoid')
        Activation function to use for output layer.
        See https://keras.io/activations/

    loss : str or obj, optional (default=keras.losses.mean_squared_error)
        String (name of objective function) or objective function.
        See https://keras.io/losses/

    optimizer : str, optional (default='adam')
        String (name of optimizer) or optimizer instance.
        See https://keras.io/optimizers/

    epochs : int, optional (default=100)
        Number of epochs to train the model.

    batch_size : int, optional (default=32)
        Number of samples per gradient update.

    dropout_rate : float in (0., 1), optional (default=0.2)
        The dropout to be used across all layers.

    l2_regularizer : float in (0., 1), optional (default=0.1)
        The regularization strength of activity_regularizer
        applied on each layer. By default, l2 regularizer is used. See
        https://keras.io/regularizers/

    validation_size : float in (0., 1), optional (default=0.1)
        The percentage of data to be used for validation.

    preprocessing : bool, optional (default=True)
        If True, apply standardization on the data.

    verbose : int, optional (default=1)
        Verbosity mode.

        - 0 = silent
        - 1 = progress bar
        - 2 = one line per epoch.

        For verbosity >= 1, model summary may be printed.

    random_state : random_state: int, RandomState instance or None, optional
        (default=None)
        If int, random_state is the seed used by the random
        number generator; If RandomState instance, random_state is the random
        number generator; If None, the random number generator is the
        RandomState instance used by `np.random`.

    contamination : float in (0., 0.5), optional (default=0.1)
        The amount of contamination of the data set, i.e.
        the proportion of outliers in the data set. When fitting this is used
        to define the threshold on the decision function.

    Attributes
    ----------
    encoding_dim_ : int
        The number of neurons in the encoding layer.

    compression_rate_ : float
        The ratio between the original feature and
        the number of neurons in the encoding layer.

    model_ : Keras Object
        The underlying AutoEncoder in Keras.

    history_: Keras Object
        The AutoEncoder training history.

    decision_scores_ : numpy array of shape (n_samples,)
        The outlier scores of the training data.
        The higher, the more abnormal. Outliers tend to have higher
        scores. This value is available once the detector is
        fitted.

    threshold_ : float
        The threshold is based on ``contamination``. It is the
        ``n_samples * contamination`` most abnormal samples in
        ``decision_scores_``. The threshold is calculated for generating
        binary outlier labels.

    labels_ : int, either 0 or 1
        The binary labels of the training data. 0 stands for inliers
        and 1 for outliers/anomalies. It is generated by applying
        ``threshold_`` on ``decision_scores_``.
    """

    def __init__(self, hidden_neurons=None,
                 hidden_activation='relu', output_activation='sigmoid',
                 epochs=3, batch_size=32, dropout_rate=0.2,
                 l2_regularizer=0.1, validation_size=0.1, preprocessing=True,
                 verbose=1, random_state=None, contamination=0.1):
        super(AutoEncoder, self).__init__(contamination=contamination)
        self.hidden_neurons = hidden_neurons
        self.hidden_activation = hidden_activation
        self.output_activation = output_activation
        self.epochs = epochs
        self.batch_size = batch_size
        self.dropout_rate = dropout_rate
        self.l2_regularizer = l2_regularizer
        self.validation_size = validation_size
        self.preprocessing = preprocessing
        self.verbose = verbose
        self.random_state = random_state

        # default values
        if self.hidden_neurons is None:
            self.hidden_neurons = [64, 32, 32, 64]

        # Verify the network design is valid
        if not self.hidden_neurons == self.hidden_neurons[::-1]:
            print(self.hidden_neurons)
            raise ValueError("Hidden units should be symmetric")

        self.hidden_neurons_ = self.hidden_neurons

        check_parameter(dropout_rate, 0, 1, param_name='dropout_rate',
                        include_left=True)
        
    def _build_model(self):
        layers = []
        layers.append(nn.Linear(self.n_features_, self.hidden_neurons[0]))
        layers.append(self._activation_fn(self.hidden_activation))
        layers.append(nn.Dropout(self.dropout_rate))

        # Hidden layers
        for i in range(1, len(self.hidden_neurons)):
            layers.append(nn.Linear(self.hidden_neurons[i - 1], self.hidden_neurons[i]))
            layers.append(self._activation_fn(self.hidden_activation))
            layers.append(nn.Dropout(self.dropout_rate))

        # Output layer
        layers.append(nn.Linear(self.hidden_neurons[-1], self.n_features_))
        layers.append(self._activation_fn(self.output_activation))

        return nn.Sequential(*layers)
    
    def recall(self, y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(self, y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    def f1(self, y_true, y_pred):
        precision = self.precision(y_true, y_pred)
        recall = self.recall(y_true, y_pred)
        return 2*((precision*recall)/(precision+recall+K.epsilon()))
    
    def _activation_fn(self, name):
        if name == 'relu':
            return nn.ReLU()
        elif name == 'sigmoid':
            return nn.Sigmoid()
        elif name == 'tanh':
            return nn.Tanh()
        else:
            raise ValueError(f"Unsupported activation: {name}")
    
    def fit(self, X, y=None,model_path='/kaggle/working/'):
        X = check_array(X)
        self.n_samples_, self.n_features_ = X.shape
        self.model_ = self._build_model().to(device)
        
        # Standardize data
        if self.preprocessing:
            self.scaler_ = StandardScaler()
            X = self.scaler_.fit_transform(X)

        # Convert to PyTorch tensors
        X_tensor = torch.tensor(X, dtype=torch.float32)
        dataset = TensorDataset(X_tensor)
#         df = dataset.to_pandas()
#         train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['out_and_tx_malicious'])
        train_dl = DataLoader(dataset, batch_size=self.batch_size, shuffle=True)
#         test_dl = DataLoader(test_df, batch_size=self.batch_size, shuffle=False)

#         optimizer = optim.AdamW(self.model_.parameters(), lr=0.001)
        optimizer = optim.Adam(self.model_.parameters(), lr=0.001, weight_decay=self.l2_regularizer)
        criterion = nn.MSELoss(reduction='mean')

#         best_loss = np.inf
#         patience_counter = 0
        
        # Custom training loop
        for epoch in range(self.epochs):
            print(f'Epoch: {epoch} started')
            train_loss = 0.0
            self.model_.train()
            loop = tqdm(train_dl, leave=True)
            for step,batch_data in enumerate(loop):
                inputs = batch_data[0].to(device)

                optimizer.zero_grad()

                outputs = self.model_(inputs)
                loss = criterion(outputs, inputs)
                loss.backward()
                optimizer.step()

                train_loss += loss.item()
                
                loop.set_description(f'Epoch {epoch}')
                if step%10000 == 0:
                    loop.set_postfix(loss=loss.item())
                
#             self.model_.eval()
#             val_loss = 0.0
#             with torch.no_grad():
#                 for batch_data in test_dl:
#                     inputs = batch_data[0].to(device)
#                     outputs = self.model_(inputs)
#                     loss = criterion(outputs, inputs)
#                     val_loss += loss.item()
                    
            train_loss /= len(train_dl)
#             val_loss /= len(test_dl)
            

            if self.verbose > 0:
                print(f"Epoch {epoch+1}/{self.epochs}, Loss: {train_loss:.4f}")
                torch.save(self.model_.state_dict(), f'{model_path}/autoenc{epoch}.pt')
            
        # Compute decision scores on training data (reconstruction error)
        if self.preprocessing:
            X_norm = self.scaler_.transform(X_tensor)
        else:
            X_norm = X_tensor.clone().to(device)
        self.model_.eval()
        with torch.no_grad():
            reconstructed = self.model_(X_norm).cpu().numpy()
        
        self.decision_scores_ = pairwise_distances_no_broadcast(X_norm.cpu().numpy(), reconstructed)
        self._process_decision_scores()

        return self
    
    def decision_function(self, X):
#         check_is_fitted(self, ['model_'])
#         X = check_array(X)
        
#         if self.preprocessing:
#             X = self.scaler_.transform(X)
        
#         X_tensor = torch.tensor(X, dtype=torch.float32)
#         self.model_.eval()
        
#         with torch.no_grad():
#             reconstructed = self.model_(X_tensor).detach().numpy()

#         return pairwise_distances_no_broadcast(X, reconstructed)
        check_is_fitted(self, ['model_', 'history_'])
        X = check_array(X)

        if self.preprocessing:
            X_norm = self.scaler_.transform(X)
        else:
            X_norm = np.copy(X)

        # Predict on X and return the reconstruction errors
        pred_scores = self.model_.predict(X_norm)
        return pairwise_distances_no_broadcast(X_norm, pred_scores)

In [16]:
epochs = 5
random_state = 42
batch_size = 256
outliers_fraction = 0.08169

lclf = AutoEncoder(hidden_neurons=[6, 4, 4, 6], 
                      hidden_activation='tanh', 
                      output_activation='sigmoid', 
                      epochs=epochs, 
                      batch_size=64, 
                      dropout_rate=0.1, 
                      l2_regularizer=0.000001, 
                      validation_size=0.20, 
                      preprocessing=False, 
                      verbose=1, 
                      random_state=random_state, 
                      contamination=outliers_fraction)
lclf.n_features_ = 7
lclf.model_ = lclf._build_model()
# model = lclf._build_model()
# model.load_state_dict(torch.load('/kaggle/input/autoencoder4/autoenc12.pt'))

In [17]:
lclf.model_.load_state_dict(torch.load('autoenc12.pt', map_location=torch.device('cpu'))) #weights_only=True
lclf.model_.eval()

C:\Users\User\AppData\Local\Temp\ipykernel_3236\2928364303.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lclf.model_.load_state_dict(torch.load('autoenc12.pt', map_loc

Sequential(
  (0): Linear(in_features=7, out_features=6, bias=True)
  (1): Tanh()
  (2): Dropout(p=0.1, inplace=False)
  (3): Linear(in_features=6, out_features=4, bias=True)
  (4): Tanh()
  (5): Dropout(p=0.1, inplace=False)
  (6): Linear(in_features=4, out_features=4, bias=True)
  (7): Tanh()
  (8): Dropout(p=0.1, inplace=False)
  (9): Linear(in_features=4, out_features=6, bias=True)
  (10): Tanh()
  (11): Dropout(p=0.1, inplace=False)
  (12): Linear(in_features=6, out_features=7, bias=True)
  (13): Sigmoid()
)

In [18]:
with torch.no_grad():
    reconstructed = lclf.model_(torch.tensor(df.to_numpy(), dtype=torch.float32)).cpu().numpy()
reconstruction_errors = pairwise_distances_no_broadcast(df.to_numpy(), reconstructed)
threshold = np.percentile(reconstruction_errors, 100 * (1 - lclf.contamination))
y_pred = (reconstruction_errors > threshold).astype(int)
eval_metric = evaluate(ytest_sample, y_pred, reconstruction_errors, [])

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

In [19]:
sample_data = np.array([10067766067, 10067750518, 20135516585, 1006776606.7, 2013550103.6, 0, 0])
x = torch.FloatTensor(sample_data).unsqueeze(0)

# Inference
with torch.no_grad():
    reconstructed = lclf.model(x)

AttributeError: 'AutoEncoder' object has no attribute 'model'